In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
# 1. Carregando Dataset e Limpando Dados
print("## 1. Importar o Conjunto de Dados")

df = pd.read_csv("Student_Performance_sujo.csv")



## 1. Importar o Conjunto de Dados


In [ ]:
# Renomeando as colunas para facilitar o acesso (remover espaços e corrigir a coluna 2)
new_columns = [
    "Hours_Studied",
    "Previous_Scores",
    "Extracurricular_Activities",
    "Sleep_Hours",
    "Sample_Question_Papers_Practiced",
    "Performance_Index"
]


df.columns = [
    'Hours Studied',
    'Previous Scores',
    'Extracurricular Activities',
    'Sleep Hours',
    'Sample Question Papers Practiced',
    'Performance Index'
]

df.columns = df.columns.str.replace(' ', '_')
print(f"Shape inicial: {df.shape}")

Shape inicial: (13310, 6)

---


In [7]:
# 2. Remoção de Duplicatas
initial_rows = len(df)
df.drop_duplicates(inplace=True)
duplicates_removed = initial_rows - len(df)
print(f"Linhas duplicadas removidas: {duplicates_removed}")
print(f"Novo Shape: {df.shape}")

Linhas duplicadas removidas: 0
Novo Shape: (11151, 6)


In [8]:
# 3. Remoção de Outliers
coluna_outlier = 'Sample_Question_Papers_Practiced'

# Convertendo a coluna para tipo numérico, forçando 'NaN' para erros de conversão
df[coluna_outlier] = pd.to_numeric(df[coluna_outlier], errors='coerce')

# Calculando o percentil 80
percentil_80 = df[coluna_outlier].quantile(0.80)

# Filtrando os outliers (removendo valores acima do percentil 80)
df_antes_outlier = len(df)
df = df[df[coluna_outlier] <= percentil_80].copy()
outliers_removed = df_antes_outlier - len(df)

print(f"Percentil 80 de '{coluna_outlier}': {percentil_80:.2f}")
print(f"Outliers removidos: {outliers_removed}")
print(f"Novo Shape: {df.shape}")

Percentil 80 de 'Sample_Question_Papers_Practiced': 9.00
Outliers removidos: 1247
Novo Shape: (9904, 6)


In [9]:
# 4. Tratamento de Valores Faltantes
print("## 4. Tratamento de Valores Faltantes (com a Média)")

# Identificando colunas numéricas que ainda podem ter NaNs e não são a coluna de texto
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()

# Excluindo 'Performance_Index' do cálculo da média de imputação 
if 'Performance_Index' in numeric_cols:
    numeric_cols.remove('Performance_Index')

for col in numeric_cols:
    if df[col].isnull().any():
        mean_value = df[col].mean()
        df[col].fillna(mean_value, inplace=True)
        print(f"Preenchidos NaNs na coluna '{col}' com a média ({mean_value:.2f}).")

# Para garantir (se houver NaNs nas colunas de 'Sleep_Hours' ou 'Previous_Scores')
if df['Sleep_Hours'].isnull().any():
    mean_sleep = df['Sleep_Hours'].mean()
    df['Sleep_Hours'].fillna(mean_sleep, inplace=True)
    print(f"Preenchidos NaNs na coluna 'Sleep_Hours' com a média ({mean_sleep:.2f}).")

if df['Previous_Scores'].isnull().any():
    mean_scores = df['Previous_Scores'].mean()
    df['Previous_Scores'].fillna(mean_scores, inplace=True)
    print(f"Preenchidos NaNs na coluna 'Previous_Scores' com a média ({mean_scores:.2f}).")

# Resetando o índice após a remoção de linhas para operações futuras
df.reset_index(drop=True, inplace=True)
print(f"Novo Shape: {df.shape}")

## 4. Tratamento de Valores Faltantes (com a Média)
Preenchidos NaNs na coluna 'Previous_Scores' com a média (69.53).
Preenchidos NaNs na coluna 'Sleep_Hours' com a média (6.54).
Novo Shape: (9904, 6)


C:\Users\rafae\AppData\Local\Temp\ipykernel_18688\3369495055.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(mean_value, inplace=True)


In [10]:
# 5. Substituindo a coluna categórica por uma numérica
print("## 5. Substituição de Coluna Categórica")

# Coluna a ser substituída é 'Extracurricular_Activities'
df['Extracurricular_Activities'] = df['Extracurricular_Activities'].str.lower().replace({'yes': 1, 'no': 0})

# Lidando com possíveis NaNs remanescentes na coluna categórica após a conversão
if df['Extracurricular_Activities'].isnull().any():
    df['Extracurricular_Activities'].fillna(0, inplace=True) # Assumindo 0 (No) para valores ausentes
    print("Preenchidos NaNs restantes em 'Extracurricular_Activities' com 0 (No).")

print("Coluna 'Extracurricular_Activities' convertida para 1 (Yes) e 0 (No).")
print(f"Novo Shape: {df.shape}")

## 5. Substituição de Coluna Categórica
Coluna 'Extracurricular_Activities' convertida para 1 (Yes) e 0 (No).
Novo Shape: (9904, 6)


C:\Users\rafae\AppData\Local\Temp\ipykernel_18688\2759377195.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Extracurricular_Activities'] = df['Extracurricular_Activities'].str.lower().replace({'yes': 1, 'no': 0})


In [11]:
# 6. Criação de Novas Colunas
print("## 6. Criação de Nova Coluna: 'practice_per_hour'")

# Garantindo que a divisão por zero não cause problemas, substituindo a divisão por zero por 0 ou NaN
df['practice_per_hour'] = np.divide(
    df['Sample_Question_Papers_Practiced'], 
    df['Hours_Studied'], 
    out=np.zeros_like(df['Sample_Question_Papers_Practiced'], dtype=float), 
    where=df['Hours_Studied'] != 0 
)

print("Nova coluna 'practice_per_hour' criada.")
print(f"Novo Shape: {df.shape}")

## 6. Criação de Nova Coluna: 'practice_per_hour'
Nova coluna 'practice_per_hour' criada.
Novo Shape: (9904, 7)


In [12]:
# 7. Normalização de Dados (Z-Score)
print("## 7. Normalização de Dados (Z-Score)")

# Excluindo a coluna alvo ('Performance_Index') e a nova coluna 'practice_per_hour' para a normalização
cols_to_normalize = ['Hours_Studied', 'Previous_Scores', 'Sleep_Hours', 'Sample_Question_Papers_Practiced', 'Extracurricular_Activities']

for col in cols_to_normalize:
    mean = df[col].mean()
    std = df[col].std()
    
    # Aplicando a normalização Z-score: z = (x - mu) / sigma
    # Garantindo que não haja divisão por zero no desvio padrão (embora improvável)
    if std != 0:
        df[col] = (df[col] - mean) / std
    else:
        # Se o desvio padrão for zero, todos os valores são iguais à média, e o Z-score é 0.
        df[col] = 0

print(f"Colunas normalizadas: {', '.join(cols_to_normalize)}")
print(f"Novo Shape: {df.shape}")

## 7. Normalização de Dados (Z-Score)
Colunas normalizadas: Hours_Studied, Previous_Scores, Sleep_Hours, Sample_Question_Papers_Practiced, Extracurricular_Activities
Novo Shape: (9904, 7)


In [17]:
# 8. Apresentar as Informações do DataFrame Resultante
# Salvar o DataFrame resultante
if df.shape == (9904, 7):
    print("**O DataFrame Final possui o Shape Correto: (9904, 7)**")
else:
    print(f"** O Shape Final está INCORRETO. Encontrado: {df.shape}, Esperado: (9904, 7)**")

print("\nPrimeiras Linhas do DataFrame:")
print(df.head())

print("\nResumo das Colunas:")
df.info()

# Salvar o DataFrame resultante
output_file = 'StudentPerformance_limpo.csv'
df.to_csv(output_file, index=False)
print(f"\nDataFrame limpo salvo como: **{output_file}**")

**O DataFrame Final possui o Shape Correto: (9904, 7)**

Primeiras Linhas do DataFrame:
   Hours_Studied  Previous_Scores  Extracurricular_Activities   Sleep_Hours  \
0       0.778655         1.848553                    1.008874 -5.700658e-16   
1      -0.379968         0.782286                   -0.991104 -1.628054e+00   
2       0.006239        -1.099361                    1.008874 -9.862169e-01   
3       0.778655         0.343235                   -0.991104  9.392942e-01   
4      -0.766176         0.531400                   -0.991104  1.581131e+00   

   Sample_Question_Papers_Practiced  Performance_Index  practice_per_hour  
0                         -1.245708               91.0           0.142857  
1                         -0.896728               65.0           0.500000  
2                         -0.896728               36.0           0.400000  
3                          0.150212               66.0           0.714286  
4                          0.499192               61.0   